In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.manifold import TSNE
from sklearn.decomposition import TruncatedSVD
import tensorflow as tf
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer, LabelEncoder, StandardScaler, normalize
from sklearn.model_selection import StratifiedKFold
import random

random.seed(10)
# load data
train= pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')

train['Data'] = 'Train'
test['Data'] = 'Test'
data = pd.concat([train, test], axis=0).reset_index(drop=True)

data_labels = data.pop('Data')
subject_labels = data.pop('subject')
labels = data.pop('Activity')

data = normalize(data)

imgsz = (17, 33)

#print ("shape: {}".format(data.shape))
#print ("type: {}".format(type(data)))

data = np.array(data)

data = data.reshape(data.shape[0], imgsz[0],imgsz[1], 1)

#print ("reshape: {}".format(data.shape))
#print ("labels: {}".format(len(labels)))

#print ("number of classes: {}".format(len(set(labels))))

enc = LabelEncoder()
labels = enc.fit_transform(labels)

kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=10)
scores = []

In [2]:
for train, test in kfold.split(data, labels): 
    model = tf.keras.Sequential()

    conv1 = tf.keras.layers.Conv2D(filters=3, kernel_size=2, data_format="channels_last", padding='same', activation='relu', input_shape=(imgsz[0], imgsz[1], 1))
    pool = tf.keras.layers.MaxPooling2D(pool_size=2)
    model.add(conv1)
    model.add(pool)
    model.add(tf.keras.layers.Dropout(0.3))
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(512, activation=tf.nn.relu))
    model.add(tf.keras.layers.Dropout(0.2))
    model.add(tf.keras.layers.Dense(256, activation=tf.nn.relu))
    model.add(tf.keras.layers.Dropout(0.2))
    model.add(tf.keras.layers.Dense(128, activation=tf.nn.relu))
    model.add(tf.keras.layers.Dropout(0.2))
    model.add(tf.keras.layers.Dense(6, activation=tf.nn.softmax))   

    model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

    #model.summary()

    history = model.fit(data[train], labels[train], epochs=20, verbose = 0)

    test_loss, test_acc = model.evaluate(data[test], labels[test]) 
    scores.append(test_acc)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
1027/1027 [==============================] - 0s 371us/sample - loss: 0.1041 - acc: 0.9630


In [3]:
print np.mean(scores)

0.95942146
